In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#from statannotations.Annotator import Annotator #https://github.com/trevismd/statannotations
from scipy.stats import friedmanchisquare

from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# if getting OptionError: "No such keys(s): 'mode.use_inf_as_null'"
# !pip install --upgrade seaborn pandas

import os
from glob import glob

if not os.path.exists('../paper/figs'):
    os.makedirs('../paper/figs')

from _load_data import * # Getting our specific labels, formatting, and data

from _cd_diagram import draw_cd_diagram

# Getting some free samples
results_df.sample(3)

# Don't change any code below, it may break (probably will) --------------------



['../models/SimplifierTopDown.py', '../models/SimplifierTopDownOnlyLast.py', '../models/SimplifierBottomUp.py', '../models/Vanilla.py', '../models/SimplifierBottomUpOnlyLast.py'] ['Top Down', 'Top Down (only last)', 'Bottom Up', 'Without simplify', 'Bottom Up (only last)']
(300, 14)
['Top Down' 'Top Down (only last)' 'Bottom Up' 'Without simplify'
 'Bottom Up (only last)']
['Airfoil' 'Concrete' 'Energy Cooling' 'Energy Heating' 'Housing' 'Tower']


,model,dataset,RunID,random_state,time,date,train_r2,test_r2,train_mse,test_mse,representation,size,complexity,depth
44,Bottom Up (only last),Airfoil,0,31658,1067.676855,01-16-2024 15:40:10,0.325526,0.376552,20.318829,22.797496,"square(add(-11.367832168828311, log1p(multiply(x_2, mul3(0.10017678773087581, sin(x_4), x_0)))))",11,2364,6
83,Without simplify,Concrete,0,16695,217.568594,01-16-2024 16:52:57,0.286632,0.486470,115.870919,102.603914,"cdiv(add4(add4(x_7, mul4(x_2, 0.0013665902157804556, cos(x_0), x_7), x_0, add(add(x_1, x_4), -185.2667916285219)), mul4(x_2, 0.08126578004531136, cos(log1p(x_0)), x_7), x_0, add(x_1, x_4)), 18.585248348435265)",28,5828,5
43,Bottom Up (only last),Airfoil,0,27690,637.778292,01-16-2024 15:38:07,0.401635,0.390722,18.930607,19.690265,"add4(mul4(add(6.563051245963081, x_1), x_0, -0.000455048576703675, sqrtabs(x_2)), -46882.395408283, -1444.265252467977, add4(multiply(x_4, -158.11550315096258), square(tan(log1p(x_3))), x_4, 48456.2903406967))",21,5262,5


criteria              Harmonic Mean size mean score                      \
                               mean              lb      mean        ub   
model                                                                     
Top Down                   3.064158        3.069579  3.433333  3.797087   
Without simplify           3.000000        2.184207  2.750000  3.315793   
Bottom Up                  2.956712        3.119849  3.550000  3.980151   
Bottom Up (only last)      2.913238        2.510232  2.816667  3.123102   
Top Down (only last)       2.842000        2.097570  2.450000  2.802430   

criteria              test_mse mean score                      
                                       lb      mean        ub  
model                                                          
Top Down                         2.359715  2.766667  3.173618  
Without simplify                 2.888874  3.300000  3.711126  
Bottom Up                        2.375993  2.533333  2.690674  
Bottom Up (only last)            2.620073  3.016667  3.413260  
Top Down (only last)             3.128203  3.383333  3.638463

criteria              Harmonic Mean complexity mean score                      \
                               mean                    lb      mean        ub   
model                                                                           
Bottom Up                  3.168832              2.990987  3.316667  3.642346   
Top Down                   3.134496              2.806610  3.183333  3.560057   
Without simplify           2.944227              2.366931  2.950000  3.533069   
Bottom Up (only last)      2.914761              2.330321  2.883333  3.436346   
Top Down (only last)       2.695172              2.425734  2.666667  2.907600   

criteria              depth mean score                     size mean score  \
                                    lb      mean        ub              lb   
model                                                                        
Bottom Up                     3.124130  3.516667  3.909204        3.119849   
Top Down                      2.889485  3.233333  3.577182        3.069579   
Without simplify              2.553090  2.833333  3.113577        2.184207   
Bottom Up (only last)         2.535772  2.950000  3.364228        2.510232   
Top Down (only last)          2.248024  2.466667  2.685310        2.097570   

criteria                                  test_mse mean score            \
                           mean        ub                  lb      mean   
model                                                                     
Bottom Up              3.550000  3.980151            2.375993  2.533333   
Top Down               3.433333  3.797087            2.359715  2.766667   
Without simplify       2.750000  3.315793            2.888874  3.300000   
Bottom Up (only last)  2.816667  3.123102            2.620073  3.016667   
Top Down (only last)   2.450000  2.802430            3.128203  3.383333   

criteria                         
                             ub  
model                            
Bottom Up              2.690674  
Top Down               3.173618  
Without simplify       3.711126  
Bottom Up (only last)  3.413260  
Top Down (only last)   3.638463

criteria              Harmonic Mean complexity mean score                      \
                               mean                    lb      mean        ub   
model                                                                           
Bottom Up                  3.168832              2.990987  3.316667  3.642346   
Top Down                   3.134496              2.806610  3.183333  3.560057   
Without simplify           2.944227              2.366931  2.950000  3.533069   
Bottom Up (only last)      2.914761              2.330321  2.883333  3.436346   
Top Down (only last)       2.695172              2.425734  2.666667  2.907600   

criteria              depth mean score                     size mean score  \
                                    lb      mean        ub              lb   
model                                                                        
Bottom Up                     3.124130  3.516667  3.909204        3.119849   
Top Down                      2.889485  3.233333  3.577182        3.069579   
Without simplify              2.553090  2.833333  3.113577        2.184207   
Bottom Up (only last)         2.535772  2.950000  3.364228        2.510232   
Top Down (only last)          2.248024  2.466667  2.685310        2.097570   

criteria                                  test_mse mean score            \
                           mean        ub                  lb      mean   
model                                                                     
Bottom Up              3.550000  3.980151            2.375993  2.533333   
Top Down               3.433333  3.797087            2.359715  2.766667   
Without simplify       2.750000  3.315793            2.888874  3.300000   
Bottom Up (only last)  2.816667  3.123102            2.620073  3.016667   
Top Down (only last)   2.450000  2.802430            3.128203  3.383333   

criteria                         
                             ub  
model                            
Bottom Up              2.690674  
Top Down               3.173618  
Without simplify       3.711126  
Bottom Up (only last)  3.413260  
Top Down (only last)   3.638463

In [2]:
def plot_rankings(columns, results_df, path):
    ranks_data = []
    for col in columns:
        ranks_df = results_df.pivot( # Smaller is better for all columns here
            index=['dataset', 'RunID', 'random_state'], columns='model', values=col
        ).rank(ascending=False, axis=1).groupby('dataset').mean()#.rank(ascending=False, axis=1)
        
        for model in results_df['model'].unique():
            df_model = ranks_df[ model ]
            values = df_model.values

            # Estimating lower and upper confidence bounds
            lb, ub = stats.norm.interval(confidence=0.95, loc=np.mean(values),
                                        scale=stats.sem(values))
            lb = np.nan_to_num(lb, 0.0)
            ub = np.nan_to_num(ub, 0.0)

            ranks_data.append( (model, f'{col} mean score', df_model.mean(), lb, ub) )

    # Creating the dataframe
    ranks_data_df = pd.DataFrame(data=ranks_data,
                                columns=['model', 'criteria', 'mean', 'lb', 'ub'])

    ranks_data_df = ranks_data_df.pivot(columns='criteria', index='model')

    # will calculate the harmonic mean and plot the columns based on mean rankings
    ranks_data_df[('mean', 'Harmonic Mean')] = (
        ranks_data_df[[('mean', f'{col} mean score') for col in columns]]
    )['mean'].apply(stats.hmean, axis=1)

    ranks_data_df = ranks_data_df.sort_values(by=('mean', 'Harmonic Mean'), ascending=False)
    ranks_data_df = ranks_data_df.reorder_levels([1, 0], axis=1).sort_index(axis=1)

    display(ranks_data_df)
    #print(ranks_data_df.to_latex())

    # Use semantically meaningful titles for the columns
    titles = ['Harmonic Mean']+[f'{col} mean score' for col in columns]

    # Now making the plot
    g = sns.PairGrid(ranks_data_df.reorder_levels([1, 0], axis=1)['mean'].reset_index(),
                    x_vars=titles, y_vars=['model'], height=2.5, aspect=1.25)

    # Draw a dot plot using the stripplot function
    g.map(sns.stripplot, size=8, orient="h", jitter=False,
        palette="flare_r", linewidth=0, edgecolor="w")

    # setting the reference as the Baseline 
    unif = ((ranks_data_df[ ranks_data_df.index=='Without simplify'])['Harmonic Mean'])['mean'].values[0]
    g.axes.flat[0].axvline(x=unif, ymin=0.025, ymax=0.975, color='gray', ls='--', zorder=-999)

    g.axes.flat[0].set(title='Harmonic Mean')
    g.axes.flat[0].set_xlabel('Better ->')
    #g.axes.flat[0].set_xlim( (-1, len(ranks_data_df.index)+1) )

    # Make the grid horizontal instead of vertical
    g.axes.flat[0].xaxis.grid(False)
    g.axes.flat[0].yaxis.grid(True)

    for ax, title in zip(g.axes.flat[1:], titles[1:]):
        color = iter(sns.color_palette("flare_r", ranks_data_df.shape[0]))
        for tick, text in zip(g.axes.flat[0].get_yticks(), g.axes.flat[0].get_yticklabels()):
            c=next(color)
            sample_name = text.get_text()  # "X" or "Y"
            single_df = ranks_data_df[ ranks_data_df.index==sample_name ]
            
            lb, ub, mean_val = (single_df[title])[ ['lb', 'ub', 'mean'] ].values[0]
            rank_val = ranks_data_df.reset_index()[(ranks_data_df.reset_index()['model']==sample_name)].index

            # plot horizontal lines across the column, centered on the tCIk
            ax.plot([lb, ub], [rank_val, rank_val], lw=3, color=c)

        # Draw a reference line on baseline
        unif = (ranks_data_df[ ranks_data_df.index=='Without simplify'])[(title, 'mean')].values[0]
        ax.axvline(x=unif, ymin=0.025, ymax=0.975, color='gray', ls='--', zorder=-999)

        # Set a different title for each axes
        ax.set(title=title)
        ax.set_xlabel('Better ->')
        #ax.set_xlim( (-1, len(ranks_data_df.index)+1) )

        # Make the grid horizontal instead of vertical
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)

    sns.despine(left=True, bottom=True)

    plt.tight_layout()
    plt.savefig(f'{path}.pdf')
    plt.savefig(f'{path}.png', dpi=600)

    plt.show()

    return ranks_data_df

plot_rankings(['size', 'test_mse'], results_df, "../paper/figs/size_mse")
plot_rankings(['size', 'complexity', 'depth', 'test_mse'], results_df, "../paper/figs/size_mse_compexity_depth")
